In [1]:
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
import os
import shutil
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from Automation import logInProjetoAlpha,logIn
import openpyxl
from openpyxl.styles import Alignment

def VerifySheetName():
    # Define the file name
    file_name = 'raspagem.xlsx'
    
    # Get the current directory path
    current_directory = os.getcwd()
    
    # Create the full path to the file
    file_path = os.path.join(current_directory, file_name)
    
    # Check if the file exists
    return os.path.isfile(file_path)


def substituir_virgula_por_ponto(nome_arquivo, nome_aba, nome_coluna, novo_nome_arquivo):
    # Carrega o workbook e a worksheet específica
    wb = openpyxl.load_workbook(nome_arquivo)
    ws = wb[nome_aba]

    # Encontrar o índice da coluna pelo nome
    col_idx = None
    for col in ws.iter_cols(1, ws.max_column):
        if col[0].value == nome_coluna:
            col_idx = col[0].column_letter
            break
    
    if not col_idx:
        raise ValueError(f"A coluna '{nome_coluna}' não foi encontrada na planilha.")
    
    # Iterar sobre as células na coluna especificada
    for cell in ws[col_idx]:
        if cell.row == 1:  # Pular o cabeçalho
            continue
        # Substituir vírgula por ponto se existir vírgula
        if ',' in str(cell.value):
            cell.value = str(cell.value).replace(',', '.')
        # Garantir que todos os valores sejam tratados como texto
        cell.value = f'="{cell.value}"' if cell.value else ''
        cell.alignment = Alignment(horizontal='left')

    # Salva o workbook atualizado
    wb.save(novo_nome_arquivo)
    print(f"Arquivo salvo como '{novo_nome_arquivo}'.")

def substituir_token(url, novo_token):
    padrao = r'token=[^&]*'
    nova_url = re.sub(padrao, f'token={novo_token}', url)
    return nova_url

def substituir_token_na_coluna( novo_token,arquivo_excel='raspagem.xlsx', coluna='Foto das Fazendas', arquivo_saida='raspagem.xlsx'):
    # Ler a planilha do Excel
  
   
    df = pd.read_excel(arquivo_excel, engine='openpyxl', dtype={'ID Fazendas': str})
    
    # Verificar se a coluna especificada existe
    if coluna not in df.columns:
        raise ValueError(f"A coluna '{coluna}' não existe no arquivo Excel.")
    
    # Função para substituir o token em uma URL
   
    
    # Aplicar a substituição na coluna especificada
    df[coluna] = df[coluna].apply(lambda x: substituir_token(x, novo_token) if isinstance(x, str) else x)
    
    # Salvar o DataFrame atualizado em um novo arquivo Excel
    df.to_excel(arquivo_excel, index=False)
    
    print(f"Arquivo salvo como {arquivo_saida}")


def TryGetImageUpdatePhotos(driver,attempts=35,delay=1):

    attempt =1
    contador = 1
    while attempt <= attempts:
        try:
            DataPage = BeautifulSoup(driver.page_source,'html.parser')
            panel = DataPage.find('question-answer')
            lista_fotos = panel.find_all('img',attrs={'class':'0 ir ir-middle'}) 
            if len(lista_fotos) == 0:
                raise Exception
            else:
                return lista_fotos[0]['src']
        except Exception as e:
            print(f'Tentando encontrar a imagem{'.'*contador}')
            contador+=1
            if contador == 5:
                contador = 1
        attempt+=1
        time.sleep(0.5)

def UpdatePhotosProjetoTerra(self,submain2_instance, download, target,chromedriver):
    


    if download == "" or target == "":
        submain2_instance.insert_text("Error! Por favor Preencha os campos adequadamente para antes de iniciar a automação.")
    elif not VerifySheetName():
        submain2_instance.insert_text('Error! Por favor coloque o nome da planilha de fotos como: raspagem.xlsx')
    else:
        options = Options()
        options.add_argument("--headless")

            #PARÂMETRO PARA MANTER WEBDRIVER ATUALIZADO
        servico = Service(ChromeDriverManager().install())

            #ENVIANDO PARÂMETROS PARA O DRIVER
        driver = webdriver.Chrome(options=options,service=servico)



        pathsWebdriver = chromedriver
        #downloadspath = str(input('Digite o caminho do download: '))
        #targetpath = str(input('Digite o caminho onde o programa está sendo executado: '))
        #pathsWebdriver = arg3
        downloadspath = download
        targetpath = target
        driver=logIn(submain2_instance,pathsWebdriver,driver)
        lista_fotos_final = []
    
        for i in range(1,2):
            #Iterate over each element on table
            
            driver.get(f'https://survey123.arcgis.com/surveys/7c7eb1d6f2b74421a50e01a2d9eb2667/data?extent=-80.6008,-21.9941,-13.8039,-9.7200&objectIds={(i)}')
            #Trying get element


            url=TryGetImageUpdatePhotos(driver)
            print(f'Url obtida para ser atualizada!')
            match = re.search(r'token=([^&]+)',url)
            if match:
                token = match.group(1)
                print('Substituindo novo token na tabela!')
                substituir_token_na_coluna(token)
            else:
                print("Token not found")


def replace_commas_with_periods_in_excel(file_path = 'raspagem.xlsx', column_name='ID Fazendas'):
    """
    Replace commas with periods in a specified column of a DataFrame loaded from an Excel file,
    and save the updated DataFrame back to the same Excel file.

    Parameters:
    file_path (str): The path to the Excel file.
    column_name (str): The name of the column to process.
    """
    # Load the DataFrame from the Excel file
    df = pd.read_excel(file_path)

    # Replace commas with periods only if the row contains a comma
    df[column_name] = df[column_name].astype(str).apply(lambda x: x.replace(',', '.') if ',' in x else x)

    # Ensure values without commas remain as strings to avoid float conversion
    df[column_name] = df[column_name].apply(lambda x: x.rstrip('.0') if '.' in x and x.endswith('.0') else x)

    # Save the updated DataFrame back to the same Excel file
    df.to_excel(file_path, index=False)

def UpdatePhotosAlphaProject(self,submain2_instance, download, target,chromedriver):

    


    if download == "" or target == "":
        submain2_instance.insert_text("Error! Por favor Preencha os campos adequadamente para antes de iniciar a automação.")
    elif not VerifySheetName():
        submain2_instance.insert_text("Error! Por favor coloque o nome da planilha de imagens como : raspagem.xlsx.")
    else:
        options = Options()
        options.add_argument("--headless")

        #PARÂMETRO PARA MANTER WEBDRIVER ATUALIZADO
        servico = Service(ChromeDriverManager().install())

        #ENVIANDO PARÂMETROS PARA O DRIVER
        driver = webdriver.Chrome(options=options,service=servico)



        pathsWebdriver = chromedriver
        #downloadspath = str(input('Digite o caminho do download: '))
        #targetpath = str(input('Digite o caminho onde o programa está sendo executado: '))
        #pathsWebdriver = arg3
        downloadspath = download
        targetpath = target
        driver=logInProjetoAlpha(pathsWebdriver,driver)

        lista_fotos_final = []
    
        for i in range(1,2):
            #Iterate over each element on table
            
            driver.get (f'https://survey123.arcgis.com/surveys/75bc7c0c6e674430b13433169665526f/data?extent=63.1750,-57.1147,51.9250,81.8793&objectIds={(i)}')
            #Trying get element

        
            url=TryGetImageUpdatePhotos(driver)
            match = re.search(r'token=([^&]+)',url)
            if match:
                token = match.group(1)
                substituir_token_na_coluna(token)
            else:
                print("Token not found")
            substituir_virgula_por_ponto('raspagem.xlsx', 'Sheet1', 'ID Fazendas', 'raspagem.xlsx')



### LogIn

In [612]:
import os
import shutil
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from Automation import logInProjetoAlpha



options = Options()
options.add_argument("--headless")

        #PARÂMETRO PARA MANTER WEBDRIVER ATUALIZADO
servico = Service(ChromeDriverManager().install())

        #ENVIANDO PARÂMETROS PARA O DRIVER
driver = webdriver.Chrome(options=options,service=servico)



pathsWebdriver = r'C:\Users\jorge\OneDrive\Desktop\UltimaVersao\chromedriver.exe'
        #downloadspath = str(input('Digite o caminho do download: '))
        #targetpath = str(input('Digite o caminho onde o programa está sendo executado: '))
        #pathsWebdriver = arg3
downloadspath = r'C:\Users\jorge\Downloads'
targetpath = r'C:\Users\jorge\OneDrive\Desktop\UltimaVersao'
driver=logInProjetoAlpha(pathsWebdriver,driver)



Nao recebeu error de verificação de email!
Nao recebeu error de verificação de email!
Nao recebeu error de verificação de email!
Nao recebeu error de verificação de email!
Nao recebeu error de verificação de email!
Passou pela fase de error na verificação de email!
Login Feito!
Finalizado o login!


### Teste tentar iterar por cada elemento da tabela BeatifulSoup

In [614]:
from bs4 import BeautifulSoup

# Supondo que você tenha o conteúdo HTML em uma variável chamada html_content

driver.get('https://survey123.arcgis.com/surveys/7c7eb1d6f2b74421a50e01a2d9eb2667/data')

In [421]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

cells = soup.find_all('vaadin-grid-cell-content')

    # Itere sobre cada célula encontrada
#cells[100].get_text(strip=True)
cells[756]

<vaadin-grid-cell-content slot="vaadin-grid-cell-content-756" title="9 de novembro de 2023">9 de novembro de 2023</vaadin-grid-cell-content>

In [615]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
actions = ActionChains(driver)


 - De 1 até 18 dá certo depois nao pegar certo (id = 206).
 - 16  id :3.383
 - 17 id:2.457

#### Código de teste manual

In [327]:
# Selecione o elemento desejado (por exemplo, o 75º elemento)
cell = cells[1188]

# Extraia o valor do atributo 'slot'
slot_value = cell.get('slot')

# Construa o seletor CSS usando o valor do atributo 'slot'
css_selector = f'vaadin-grid-cell-content[slot="{slot_value}"]'

# Use Selenium para esperar e encontrar o elemento, e então clicar nele
try:
    element_to_click = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
    )
    element_to_click.click()
    actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
    #actions.move_to_element(element_to_click ).send_keys(Keys.PAGE_DOWN).perform()
    print('Deu certo')
finally:
    print('')

IndexError: list index out of range

#### Código de teste automático

In [301]:
actions = ActionChains(driver)

In [361]:
def scroll_down(driver, pixels):
    driver.execute_script(f"window.scrollBy(0, {pixels});")

# Dê um pouco de scroll para baixo (ajuste o valor de pixels conforme necessário)


In [436]:
import time
for i in range(0,25):
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        cells = soup.find_all('vaadin-grid-cell-content')
        cell = cells[108+i*36]
        # Extraia o valor do atributo 'slot'
        slot_value = cell.get('slot')

        # Construa o seletor CSS usando o valor do atributo 'slot'
        css_selector = f'vaadin-grid-cell-content[slot="{slot_value}"]'
        time.sleep(2)
        # Use Selenium para esperar e encontrar o elemento, e então clicar nele
        try:
            element_to_click = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
            )
            element_to_click.click()

                 
            #actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
            #actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
            actions.move_to_element(element_to_click ).send_keys(Keys.PAGE_DOWN).perform()
            print(i)

        except Exception as e:
              for i in range(0,25):
                    cell = cells[108+i*36]
                    # Extraia o valor do atributo 'slot'
                    slot_value = cell.get('slot')

                    # Construa o seletor CSS usando o valor do atributo 'slot'
                    css_selector = f'vaadin-grid-cell-content[slot="{slot_value}"]'
                    time.sleep(2)
                    # Use Selenium para esperar e encontrar o elemento, e então clicar nele
                    try:
                        element_to_click = WebDriverWait(driver, 100).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
                        )
                        element_to_click.click()

                            
                        #actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
                        actions.move_to_element(element_to_click ).send_keys(Keys.PAGE_DOWN).perform()
                        #actions.move_to_element(element_to_click ).send_keys(Keys.PAGE_DOWN).perform()
                        print(i)
                    except Exception as e:
                        print(e)
              

0
1
2
3
4


KeyboardInterrupt: 

#### Código fazendo iteração de (0,25) e iterando com um loop j por fora

In [437]:
for i in range(1,20):
    for j in range(0,25):
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        cells = soup.find_all('vaadin-grid-cell-content')
        cell = cells[108+j*36]
        # Extraia o valor do atributo 'slot'
        slot_value = cell.get('slot')

        # Construa o seletor CSS usando o valor do atributo 'slot'
        css_selector = f'vaadin-grid-cell-content[slot="{slot_value}"]'
        time.sleep(2)
        # Use Selenium para esperar e encontrar o elemento, e então clicar nele
        try:
            element_to_click = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
            )
            element_to_click.click()

                 
            #actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
            #actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
            actions.move_to_element(element_to_click ).send_keys(Keys.PAGE_DOWN).perform()
            print(j)
        except:
            print('')

    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17






KeyboardInterrupt: 

In [455]:
from DownloadPhotos import TentarPegarElementoModificadoNovo,AddArrayDataframe,verificar_arquivo_excel,delete_excel_file,adicionar_dados_dataframe,getTotalValues


def BaixarFotosdaTabela(driver):

    lista_fotos_final = []
    lista_contador_final = []

    cidades_support = pd.DataFrame(columns=["Cidade", "ID Fazendas"])
    _Fotos = pd.DataFrame(columns=["Foto das Fazendas"])
    _contador = pd.DataFrame(columns=['Contador'])
    raspagem = pd.DataFrame(columns=['Contador','Cidade', 'ID Fazendas', 'Foto das Fazendas'])
    qtdLinhas = 1

    if verificar_arquivo_excel('raspagem.xlsx'):
        delete_excel_file('raspagem.xlsx')
        raspagem.to_excel('raspagem.xlsx', index=False)
    else:
        #Caso o arquivo excel nao exista crie ele
        raspagem.to_excel('raspagem.xlsx', index=False)
    totalValues = getTotalValues(driver)
        
    while qtdLinhas <= totalValues:
        qtdmaxlinhas=0
        while qtdmaxlinhas <= 24:
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            cells = soup.find_all('vaadin-grid-cell-content')
            cell = cells[108+qtdmaxlinhas*36]
            # Extraia o valor do atributo 'slot'
            slot_value = cell.get('slot')

             # Construa o seletor CSS usando o valor do atributo 'slot'
            css_selector = f'vaadin-grid-cell-content[slot="{slot_value}"]'
            time.sleep(5)
            # Use Selenium para esperar e encontrar o elemento, e então clicar nele
     
            element_to_click = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
            )
            element_to_click.click()

                 
            actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
                #actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
            #actions.move_to_element(element_to_click ).send_keys(Keys.PAGE_DOWN).perform()

            """
            Em diante teremos o código que irá pegar os dados
            """
            try:
                _cidade, answer_text = TentarPegarElementoModificadoNovo(driver, retries=45)
                print('TryGetElement complete: Element its avaible for to get!')
            except Exception as e:
                print(f'Error during TryGetElement: {e}')
                continue
            DataPage = BeautifulSoup(driver.page_source, 'html.parser')
            panel = DataPage.find('question-answer')

            lista_fotos = panel.find_all('img', attrs={'class': '0 ir ir-middle'})
                
            for foto in lista_fotos:
                data_city = {_cidade: answer_text}
                        #print(data_city)
                cidades_df = pd.DataFrame(list(data_city.items()), columns=["Cidade", "ID Fazendas"])
                        #print(data_city.items())
                cidades_support = pd.concat([cidades_support, cidades_df], ignore_index=True)
                        #print(cidades_support)
                lista_fotos_final.append(foto['src'])
                lista_contador_final.append(qtdLinhas)


            print(len(lista_fotos_final))

                    # Criar a planilha do Excel toda vez que o programa rodar

            AddArrayDataframe(_Fotos, lista_fotos_final)
            AddArrayDataframe(_contador,lista_contador_final)
                    #print(_Fotos)
            print('Fotos dataframe:',_Fotos)
                    #Criando o dataframe do contador-------------------------------------------------------------------------------
            contador = {qtdLinhas: answer_text}
            contador_df = pd.DataFrame(list(contador.items()), columns=["Contador", "ID Fazendas"])

            final_Data = cidades_support.join(_Fotos)
            final_Data = final_Data.join(_contador)
            print('DataFrame Final')
            
            final_Data = final_Data[['Contador', 'Cidade', 'ID Fazendas', 'Foto das Fazendas']]
            print(final_Data)
            adicionar_dados_dataframe('raspagem.xlsx', final_Data)

                    # Esvaziar DataFrames
            contador_df = contador_df.iloc[0:0]
            cidades_support = cidades_support.iloc[0:0]
            _Fotos = _Fotos.iloc[0:0]
            final_Data=final_Data.iloc[0:0]
            lista_contador_final = []
            lista_fotos = []
            lista_fotos_final = []

            qtdmaxlinhas=qtdmaxlinhas+1
            qtdLinhas+=1
        
        print(f'Linha adicionada {qtdLinhas}')
        
BaixarFotosdaTabela(driver)
    

Fazenda:4.182
Encontrou as fotos na página, quantidade de fotos: 4!
Cidade obtida na imagem:Veríssimo
Cidade encontrada na Imagem. Cidade: Veríssimo
TryGetElement complete: Element its avaible for to get!


TypeError: object of type 'WebElement' has no len()

In [453]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

cells = soup.find_all('vaadin-grid-cell-content')
cell = cells[108+1*36]
            # Extraia o valor do atributo 'slot'
slot_value = cell.get('slot')

             # Construa o seletor CSS usando o valor do atributo 'slot'
css_selector = f'vaadin-grid-cell-content[slot="{slot_value}"]'
time.sleep(2)
            # Use Selenium para esperar e encontrar o elemento, e então clicar nele
     
element_to_click = WebDriverWait(driver, 100).until(
EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
)
element_to_click.click()

                 
actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
_cidade, answer_text = TentarPegarElementoModificadoNovo(driver, retries=45)
print('TryGetElement complete: Element its avaible for to get!')
print(_cidade, answer_text )
            

Fazenda:3.290
Imagens ainda não encontrada .
Imagens ainda não encontrada ..
Imagens ainda não encontrada ...
Imagens ainda não encontrada ....
Imagens ainda não encontrada .....
Imagens ainda não encontrada .
Imagens ainda não encontrada ..
Imagens ainda não encontrada ...
Encontrou as fotos na página, quantidade de fotos: 3!
Cidade obtida na imagem:Veríssimo
Cidade encontrada na Imagem. Cidade: Veríssimo
TryGetElement complete: Element its avaible for to get!
Veríssimo 3.290


In [466]:
lista_fotos_final = []
lista_contador_final = []

cidades_support = pd.DataFrame(columns=["Cidade", "ID Fazendas"])
_Fotos = pd.DataFrame(columns=["Foto das Fazendas"])
_contador = pd.DataFrame(columns=['Contador'])
raspagem = pd.DataFrame(columns=['Contador','Cidade', 'ID Fazendas', 'Foto das Fazendas'])
qtdLinhas = 1
cidades = []
idFazenda = []

if verificar_arquivo_excel('raspagem.xlsx'):
    delete_excel_file('raspagem.xlsx')
    raspagem.to_excel('raspagem.xlsx', index=False)
else:
    #Caso o arquivo excel nao exista crie ele
    raspagem.to_excel('raspagem.xlsx', index=False)
totalValues = getTotalValues(driver)
        
while qtdLinhas <= 1:
    qtdmaxlinhas=0
    while qtdmaxlinhas <= 3:
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        cells = soup.find_all('vaadin-grid-cell-content')
        cell = cells[108+qtdmaxlinhas*36]
            # Extraia o valor do atributo 'slot'
        slot_value = cell.get('slot')

             # Construa o seletor CSS usando o valor do atributo 'slot'
        css_selector = f'vaadin-grid-cell-content[slot="{slot_value}"]'
        time.sleep(5)
            # Use Selenium para esperar e encontrar o elemento, e então clicar nele
     
        element_to_click = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
        )
        element_to_click.click()

                 
        #actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
                #actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
        

        """
        Em diante teremos o código que irá pegar os dados
        """
        try:
            _cidade, answer_text = TentarPegarElementoModificadoNovo(driver, retries=45)
            print('TryGetElement complete: Element its avaible for to get!')
        except Exception as e:
            print(f'Error during TryGetElement: {e}')
            continue
        
        cidades.append(_cidade)
        idFazenda.append(answer_text)
        qtdmaxlinhas=qtdmaxlinhas+1
        qtdLinhas+=1
        time.sleep(2)
        actions.move_to_element(element_to_click ).send_keys(Keys.PAGE_DOWN).perform()
    
        print(f'Linha adicionada {qtdLinhas}')

Fazenda:12.407
Encontrou as fotos na página, quantidade de fotos: 2!
Não conseguiu encontrar as imagems na página:Message: element not interactable
  (Session info: chrome=126.0.6478.62)
Stacktrace:
	GetHandleVerifier [0x00007FF792EC1F52+60322]
	(No symbol) [0x00007FF792E3CEC9]
	(No symbol) [0x00007FF792CF7CE9]
	(No symbol) [0x00007FF792D489C2]
	(No symbol) [0x00007FF792D3B491]
	(No symbol) [0x00007FF792D6C21A]
	(No symbol) [0x00007FF792D3ADB6]
	(No symbol) [0x00007FF792D6C430]
	(No symbol) [0x00007FF792D8BC80]
	(No symbol) [0x00007FF792D6BFC3]
	(No symbol) [0x00007FF792D39617]
	(No symbol) [0x00007FF792D3A211]
	GetHandleVerifier [0x00007FF7931D94AD+3301629]
	GetHandleVerifier [0x00007FF7932236D3+3605283]
	GetHandleVerifier [0x00007FF793219450+3563680]
	GetHandleVerifier [0x00007FF792F74326+790390]
	(No symbol) [0x00007FF792E4750F]
	(No symbol) [0x00007FF792E43404]
	(No symbol) [0x00007FF792E43592]
	(No symbol) [0x00007FF792E32F9F]
	BaseThreadInitThunk [0x00007FF9C8757344+20]
	RtlUserT

In [470]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

cells = soup.find_all('vaadin-grid-cell-content')
cell = cells[108+2*36]
            # Extraia o valor do atributo 'slot'
slot_value = cell.get('slot')

             # Construa o seletor CSS usando o valor do atributo 'slot'
css_selector = f'vaadin-grid-cell-content[slot="{slot_value}"]'
            # Use Selenium para esperar e encontrar o elemento, e então clicar nele
     
element_to_click = WebDriverWait(driver, 100).until(
EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
)
element_to_click.click()

#### Obter fazendas

In [623]:
def Carregando(texto):
    contador = 1
    while contador <= 5:
        print(f'{texto} {'.'*contador}')
        time.sleep(0.5)
        contador=contador+1

In [568]:

import time
def PegarFazenda(driver,tentativas=65,delay=0.5):
    DataPage = BeautifulSoup(driver.page_source, 'html.parser')
    # Encontrar o elemento <p> que contém um <b> com o texto 'ID Fazenda'
    element = DataPage.find('p', string=lambda t: t and 'ID Fazenda' in t)  
    while element == None:
        DataPage = BeautifulSoup(driver.page_source, 'html.parser')
        # Encontrar o elemento <p> que contém um <b> com o texto 'ID Fazenda'
        element = DataPage.find('p', string=lambda t: t and 'ID Fazenda' in t)
        Carregando('Tentando Pegar Fazenda')
    answer_element = element.find_next('p', class_='answer')
    answer_text = answer_element.get_text()
    return answer_text

#PegarFazenda(driver)



### Obter imagens

In [541]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Encontrar todos os elementos <img> com a classe '0 ir ir-middle'
image_elements = soup.find_all('img', class_='0 ir ir-middle')
print(image_elements)
# Extrair os URLs das imagens
image_urls = [img['src'] for img in image_elements]

# Exibir os URLs das imagens
for url in image_urls:
    print(url)

[<img _ngcontent-xsg-c33="" class="0 ir ir-middle" src="https://services7.arcgis.com/3S53oeHtMJ8uVqCM/arcgis/rest/services/service_737f4333f2764202955b5c9e0b023ea0/FeatureServer/1/1/attachments/1?token=3NKHt6i2urmWtqOuugvr9eqCAVdLUU4nGBBXSgOaRvKQh02UjnnB7j3qYiOjHkuWzEPjI82Jgds4RCK82O5in4MVAtpMbcaij7mPKLR2wepR01-OO801YeQ-DgHgBNAVS2MREKYqoYwdfudAq5RMJQxkaGa4dzFCM_rqi-P_oXTFoHaRd1u7GGW2FFWc3ArzicO-ql125jXOcfxbEyRQQu63hJPXz-LJlV43VIW2mkY.&amp;w=400"/>, <img _ngcontent-xsg-c33="" class="0 ir ir-middle" src="https://services7.arcgis.com/3S53oeHtMJ8uVqCM/arcgis/rest/services/service_737f4333f2764202955b5c9e0b023ea0/FeatureServer/1/2/attachments/2?token=3NKHt6i2urmWtqOuugvr9eqCAVdLUU4nGBBXSgOaRvKQh02UjnnB7j3qYiOjHkuWzEPjI82Jgds4RCK82O5in4MVAtpMbcaij7mPKLR2wepR01-OO801YeQ-DgHgBNAVS2MREKYqoYwdfudAq5RMJQxkaGa4dzFCM_rqi-P_oXTFoHaRd1u7GGW2FFWc3ArzicO-ql125jXOcfxbEyRQQu63hJPXz-LJlV43VIW2mkY.&amp;w=400"/>, <img _ngcontent-xsg-c33="" class="0 ir ir-middle" src="https://services7.arcgis.com/3S53oeHtMJ8

In [512]:
def PegarImagensNaPagina(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    imagens = []
    # Encontrar todos os elementos <img> com a classe '0 ir ir-middle'
    image_elements = soup.find_all('img', class_='0 ir ir-middle')
    while len(image_elements)<=0:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        image_elements = soup.find_all('img', class_='0 ir ir-middle')
        Carregando('Não encontrou as imagens')
    image_urls = [img['src'] for img in image_elements]

    # Exibir os URLs das imagens
    for url in image_urls:
        print(url)
        imagens.append(url)
        
    return imagens
    
PegarImagensNaPagina(driver)
    



Não encontrou as imagens .
Não encontrou as imagens ..
Não encontrou as imagens ...
Não encontrou as imagens ....
Não encontrou as imagens .....
Não encontrou as imagens .
Não encontrou as imagens ..
Não encontrou as imagens ...
Não encontrou as imagens ....
Não encontrou as imagens .....
https://services7.arcgis.com/3S53oeHtMJ8uVqCM/arcgis/rest/services/service_737f4333f2764202955b5c9e0b023ea0/FeatureServer/1/637/attachments/636?token=3NKHt6i2urmWtqOuugvr9eqCAVdLUU4nGBBXSgOaRvKQh02UjnnB7j3qYiOjHkuWzEPjI82Jgds4RCK82O5in6qOp_boegx25z-g167FnoZOIXsKJt8w8fE06mlKLqu-azjLTpLhLijEn3mQD9BroskAT95cbjZtdAX5laJHrkdWGp-uTMYScSTbNXWaTWW2sAbzakr-qIh9oKtKjOSiIviTekWi1-izewLIRcbxOkY.&w=400
https://services7.arcgis.com/3S53oeHtMJ8uVqCM/arcgis/rest/services/service_737f4333f2764202955b5c9e0b023ea0/FeatureServer/1/638/attachments/637?token=3NKHt6i2urmWtqOuugvr9eqCAVdLUU4nGBBXSgOaRvKQh02UjnnB7j3qYiOjHkuWzEPjI82Jgds4RCK82O5in6qOp_boegx25z-g167FnoZOIXsKJt8w8fE06mlKLqu-azjLTpLhLijEn3mQD9BroskAT95cbjZtdAX5laJ

['https://services7.arcgis.com/3S53oeHtMJ8uVqCM/arcgis/rest/services/service_737f4333f2764202955b5c9e0b023ea0/FeatureServer/1/637/attachments/636?token=3NKHt6i2urmWtqOuugvr9eqCAVdLUU4nGBBXSgOaRvKQh02UjnnB7j3qYiOjHkuWzEPjI82Jgds4RCK82O5in6qOp_boegx25z-g167FnoZOIXsKJt8w8fE06mlKLqu-azjLTpLhLijEn3mQD9BroskAT95cbjZtdAX5laJHrkdWGp-uTMYScSTbNXWaTWW2sAbzakr-qIh9oKtKjOSiIviTekWi1-izewLIRcbxOkY.&w=400',
 'https://services7.arcgis.com/3S53oeHtMJ8uVqCM/arcgis/rest/services/service_737f4333f2764202955b5c9e0b023ea0/FeatureServer/1/638/attachments/637?token=3NKHt6i2urmWtqOuugvr9eqCAVdLUU4nGBBXSgOaRvKQh02UjnnB7j3qYiOjHkuWzEPjI82Jgds4RCK82O5in6qOp_boegx25z-g167FnoZOIXsKJt8w8fE06mlKLqu-azjLTpLhLijEn3mQD9BroskAT95cbjZtdAX5laJHrkdWGp-uTMYScSTbNXWaTWW2sAbzakr-qIh9oKtKjOSiIviTekWi1-izewLIRcbxOkY.&w=400',
 'https://services7.arcgis.com/3S53oeHtMJ8uVqCM/arcgis/rest/services/service_737f4333f2764202955b5c9e0b023ea0/FeatureServer/1/639/attachments/638?token=3NKHt6i2urmWtqOuugvr9eqCAVdLUU4nGBBXSgOaRvKQh02UjnnB7j

#### Obter Cidade

In [542]:
def VerificarGPSnaPágina(driver,tentativas = 5,delay=0.5):
    tentou = 1
    while tentou <= tentativas:
        try:
            driver.find_element(By.CLASS_NAME, "icon-gps")
            return True
        except Exception as e:
            print(e)
        tentou+=1
        time.sleep(delay)
   


In [572]:
def PegarCidadeNaPagina(driver):
    # Aguardar que os elementos <p> estejam presentes na página
    try:
        # Localizar todos os elementos <p> com a classe main-info
        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.exif-content div.item p.main-info"))
        )

        # Pegar o último elemento da lista
        last_element = elements[-1]

        # Obter o texto do último elemento <p>
        text = last_element.text

        # Dividir o texto pelas vírgulas e obter o texto entre a primeira e a segunda vírgula
        parts = text.split(',')
        if len(parts) >= 2:
            desired_text = parts[1].strip()
            return desired_text
        else:
            print("Não foi possível encontrar o texto desejado.")
    except Exception as e:
        print("Elemento não encontrado ou outra exceção ocorreu:", e)

In [586]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [560]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Encontrar todos os elementos <img> com a classe '0 ir ir-middle'
image_elements = soup.find_all('img', class_='0 ir ir-middle')
for img in image_elements:
        src = img['src']
        print(f"Clicando na imagem com src: {src}")
        
        # Encontrar o elemento <img> usando o Selenium
        img_element = driver.find_element(By.XPATH, f"//img[@src='{src}']")
        
        # Verificar se o elemento está presente e visível antes de clicar
        WebDriverWait(driver, 10).until(EC.visibility_of(img_element))
        time.sleep(2)
        # Usar ActionChains para clicar na imagem
        ActionChains(driver).move_to_element(img_element).click().perform()
        if VerificarGPSnaPágina(driver):
                print('Cidade está na página de imagem!!')
                time.sleep(2)
                driver.find_element(By.XPATH,'/html/body/s123-app/div/main/ng-component/div/ng-component/survey123-ui-media-viewer/div/div[1]/div[1]/div/i').click()
                PegarCidadeNaPagina(driver)
                break
        else:
                driver.find_element(By.XPATH,'/html/body/s123-app/div/main/ng-component/div/ng-component/survey123-ui-media-viewer/div/div[1]/div[1]/div/i').click()
                time.sleep(2)
                continue


Clicando na imagem com src: https://services7.arcgis.com/3S53oeHtMJ8uVqCM/arcgis/rest/services/service_737f4333f2764202955b5c9e0b023ea0/FeatureServer/1/177/attachments/176?token=3NKHt6i2urmWtqOuugvr9eqCAVdLUU4nGBBXSgOaRvKQh02UjnnB7j3qYiOjHkuWzEPjI82Jgds4RCK82O5in0h1W6hpt5tYbspz3NOnoog1zUE3huf5ArTW7JjWrbj6T18NOLCXZGBhNS5n7PwjqMgm6VwfHkAPEZ6_nJuHA8Vkn7R78K4IQSnfaU3NAR5SPilQUPttyWriskVnO7fjD0GY7hig-U27kbgy-_S2jKg.&w=400
Cidade está na página de imagem!!


KeyboardInterrupt: 

In [573]:


def IterarImagensNaPagina(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    cidade = ''
    # Encontrar todos os elementos <img> com a classe '0 ir ir-middle'
    image_elements = soup.find_all('img', class_='0 ir ir-middle')
    for img in image_elements:
        src = img['src']
        
        # Encontrar o elemento <img> usando o Selenium
        img_element = driver.find_element(By.XPATH, f"//img[@src='{src}']")
        
        # Verificar se o elemento está presente e visível antes de clicar
        WebDriverWait(driver, 10).until(EC.visibility_of(img_element))
        time.sleep(3)
        
        # Usar ActionChains para clicar na imagem
        ActionChains(driver).move_to_element(img_element).click().perform()
        
        if VerificarGPSnaPágina(driver):
            cidade=PegarCidadeNaPagina(driver)
            time.sleep(2)
            driver.find_element(By.XPATH, '/html/body/s123-app/div/main/ng-component/div/ng-component/survey123-ui-media-viewer/div/div[1]/div[1]/div/i').click()
            return cidade
        else:
            driver.find_element(By.XPATH, '/html/body/s123-app/div/main/ng-component/div/ng-component/survey123-ui-media-viewer/div/div[1]/div[1]/div/i').click()
            time.sleep(2)
            continue

    return cidade

#IterarImagensNaPagina(driver)

#### Jutando tudo

In [608]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [609]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Navegue para a URL desejada
driver.get('https://survey123.arcgis.com/surveys/7c7eb1d6f2b74421a50e01a2d9eb2667/data?extent=-80.6008,-22.0552,-13.8039,-9.6550&objectIds=1')

In [627]:

qtdLinhas=1      
qtdmaxlinhas=0 
idfazendas = []

while qtdLinhas <= 221:

    try:
        print(f'-------------------------Fazenda n {qtdLinhas}-----------------------')
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        cells = soup.find_all('vaadin-grid-cell-content')
        print(qtdmaxlinhas)
        cell = cells[108+qtdmaxlinhas*36]
            # Extraia o valor do atributo 'slot'
        slot_value = cell.get('slot')

             # Construa o seletor CSS usando o valor do atributo 'slot'
        css_selector = f'vaadin-grid-cell-content[slot="{slot_value}"]'
        time.sleep(1)
            # Use Selenium para esperar e encontrar o elemento, e então clicar nele
     
        element_to_click = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
         )
        element_to_click.click()
        time.sleep(1)
                 
        actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
            #actions.move_to_element(element_to_click ).click().send_keys(Keys.ARROW_DOWN).perform()
            #actions.move_to_element(element_to_click ).send_keys(Keys.PAGE_DOWN).perform()
        
        
        """
        Em diante teremos o código que irá pegar os dados
        """
        id = PegarFazenda(driver)
        print(id)
        idfazendas.append(id)
        qtdmaxlinhas+=1
        qtdLinhas+=1
    except Exception as e:
        print(e)
        actions.move_to_element(element_to_click ).send_keys(Keys.PAGE_UP).perform()
        qtdmaxlinhas = 0



print(idfazendas)




    

-------------------------Fazenda n 1-----------------------
0
Tentando Pegar Fazenda .
Tentando Pegar Fazenda ..
Tentando Pegar Fazenda ...
Tentando Pegar Fazenda ....
Tentando Pegar Fazenda .....
Tentando Pegar Fazenda .
Tentando Pegar Fazenda ..
Tentando Pegar Fazenda ...
Tentando Pegar Fazenda ....
Tentando Pegar Fazenda .....
12.407
-------------------------Fazenda n 2-----------------------
1
Tentando Pegar Fazenda .
Tentando Pegar Fazenda ..
Tentando Pegar Fazenda ...
Tentando Pegar Fazenda ....
Tentando Pegar Fazenda .....
Tentando Pegar Fazenda .
Tentando Pegar Fazenda ..
Tentando Pegar Fazenda ...
Tentando Pegar Fazenda ....
Tentando Pegar Fazenda .....
6.311
-------------------------Fazenda n 3-----------------------
2
Tentando Pegar Fazenda .
Tentando Pegar Fazenda ..
Tentando Pegar Fazenda ...
Tentando Pegar Fazenda ....
Tentando Pegar Fazenda .....
Tentando Pegar Fazenda .
Tentando Pegar Fazenda ..
Tentando Pegar Fazenda ...
Tentando Pegar Fazenda ....
Tentando Pegar Fazen

KeyboardInterrupt: 

In [598]:
print(f'-------------------------Fazenda n {qtdLinhas}-----------------------')
soup = BeautifulSoup(driver.page_source, 'html.parser')

cells = soup.find_all('vaadin-grid-cell-content')
cell = cells[108+1*36]
            # Extraia o valor do atributo 'slot'
slot_value = cell.get('slot')

             # Construa o seletor CSS usando o valor do atributo 'slot'
css_selector = f'vaadin-grid-cell-content[slot="{slot_value}"]'

-------------------------Fazenda n 1-----------------------


In [599]:
element_to_click = WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
         )
element_to_click.click()
time.sleep(1)